In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from ot_pert.metrics import compute_mean_metrics, compute_metrics

In [2]:
adata_preds = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions.h5ad")

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [3]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_300.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_300.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_300.h5ad"

In [4]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [5]:
adata_preds.obsm

AxisArrays with keys: CPA_pred, perts, perts_doses

In [6]:
def reconstruct_data(embedding: np.ndarray, projection_matrix: np.ndarray, mean_to_add: np.ndarray) -> np.ndarray:
    return np.matmul(embedding, projection_matrix.T) + mean_to_add


def project_data(data: np.ndarray, projection_matrix: np.ndarray, mean_to_subtract: np.ndarray) -> np.ndarray:
    return np.matmul(data - mean_to_subtract, projection_matrix)

In [7]:
OBSM_KEY_DATA_EMBEDDING = "X_pca"

In [8]:
project_data_fn = functools.partial(
    project_data,
    projection_matrix=adata_train.varm["PCs"],
    mean_to_subtract=adata_train.varm["X_train_mean"].T,
)

In [9]:
adata_preds

AnnData object with n_obs × n_vars = 67382 × 2000
    obs: 'condition', 'cell_type', 'condition_ID', 'log_dose', 'smiles_rdkit', 'split', 'split_1ct_MEC', 'CPA_cat', 'CPA_CHEMBL504', '_scvi_condition_ID', '_scvi_cell_type', '_scvi_CPA_cat'
    uns: '_scvi_manager_uuid', '_scvi_uuid'
    obsm: 'CPA_pred', 'perts', 'perts_doses'

In [10]:
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm[OBSM_KEY_DATA_EMBEDDING]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.A
    pred_cpa = adata_preds[adata_preds.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_decoded_predicted[cond] = pred_cpa
    test_data_target_encoded_predicted[cond] = project_data_fn(pred_cpa)


ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm[OBSM_KEY_DATA_EMBEDDING]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.A
    pred_cpa = adata_preds[adata_preds.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_decoded_predicted[cond] = pred_cpa
    ood_data_target_encoded_predicted[cond] = project_data_fn(pred_cpa)

In [11]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [12]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [13]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [14]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [15]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [16]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7391917749232417,
 'decoded_ood_sinkhorn_div_1': 103.79780883789063,
 'decoded_ood_sinkhorn_div_10': 72.30388488769532,
 'decoded_ood_sinkhorn_div_100': 27.833563232421874,
 'decoded_ood_e_distance': 54.769449029362136,
 'decoded_ood_mmd': 0.246560040331874}

In [17]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.956132341509689,
 'decoded_test_sinkhorn_div_1': 82.31482153672438,
 'decoded_test_sinkhorn_div_10': 63.47894507188063,
 'decoded_test_sinkhorn_div_100': 6.181912348820613,
 'decoded_test_e_distance': 8.298093321851823,
 'decoded_test_mmd': 0.1823629892674941}

In [18]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7391917749232417,
 'decoded_ood_sinkhorn_div_1': 103.79780883789063,
 'decoded_ood_sinkhorn_div_10': 72.30388488769532,
 'decoded_ood_sinkhorn_div_100': 27.833563232421874,
 'decoded_ood_e_distance': 54.769449029362136,
 'decoded_ood_mmd': 0.246560040331874}

In [19]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.5159547481300902,
 'deg_ood_sinkhorn_div_1': 24.493817806243896,
 'deg_ood_sinkhorn_div_10': 16.49903392791748,
 'deg_ood_sinkhorn_div_100': 15.873676109313966,
 'deg_ood_e_distance': 31.61525838731057,
 'deg_ood_mmd': 0.34742090103299406}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.956132341509689,
 'decoded_test_sinkhorn_div_1': 82.31482153672438,
 'decoded_test_sinkhorn_div_10': 63.47894507188063,
 'decoded_test_sinkhorn_div_100': 6.181912348820613,
 'decoded_test_e_distance': 8.298093321851823,
 'decoded_test_mmd': 0.1823629892674941}

In [21]:
mean_test_metrics_encoded

{'encoded_test_r_squared': 0.7785426383460113,
 'encoded_test_sinkhorn_div_1': 53.68342032799354,
 'encoded_test_sinkhorn_div_10': 34.97506611163799,
 'encoded_test_sinkhorn_div_100': 4.564601017878606,
 'encoded_test_e_distance': 7.258525984396837,
 'encoded_test_mmd': 0.16077130219967567}

In [22]:
ood_metrics_encoded

{'Cediranib+PCI-34051': {'r_squared': 0.9074379652405279,
  'sinkhorn_div_1': 44.772613525390625,
  'sinkhorn_div_10': 14.927448272705078,
  'sinkhorn_div_100': 1.5101356506347656,
  'e_distance': 2.6157738667272463,
  'mmd': 0.17343959115221655},
 'Givinostat+SRT1720': {'r_squared': 0.8628172793421973,
  'sinkhorn_div_1': 46.553855895996094,
  'sinkhorn_div_10': 16.30636978149414,
  'sinkhorn_div_100': 1.3358192443847656,
  'e_distance': 2.271262392813491,
  'mmd': 0.16894391608478376},
 'Panobinostat+Crizotinib': {'r_squared': -0.9476498324506673,
  'sinkhorn_div_1': 115.91564178466797,
  'sinkhorn_div_10': 83.86618041992188,
  'sinkhorn_div_100': 68.12455749511719,
  'e_distance': 135.32155961493643,
  'mmd': 0.32815091943184865},
 'Panobinostat+PCI-34051': {'r_squared': -1.1604597817810318,
  'sinkhorn_div_1': 105.5364761352539,
  'sinkhorn_div_10': 73.5385513305664,
  'sinkhorn_div_100': 58.305912017822266,
  'e_distance': 115.77804275511745,
  'mmd': 0.3094106159503713},
 'SRT210

In [23]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/cpa"

In [25]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, "ood_metrics_encoded.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, "ood_metrics_decoded.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, "test_metrics_encoded.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, "test_metrics_decoded.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, "test_metrics_deg.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, "ood_metrics_ood.csv"))
